# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
####Just import everything I already listed in the 00-setup notebook#### 

import json
from glob import glob

# do use pathlib as we are downloading and calling data for this assignment

from pathlib import Path
import os

# import earthpy and pandas as pd & geopandas as gpd. Also geopandas is useful for working with the zipfiles.
import earthpy as py
import pandas as pd 
import geopandas as gpd
#To help with plotting we will need the following packages for raster data. Hvplot.xarray allows for interactive ploting. Matplotlib packages can help make our final figures look much nicer, so we can include those as well. 

#Packages for plotting and working with raster data (remember we are using pandas and not earthpy):
import hvplot.xarray
import hvplot.pandas
import rioxarray as rxr
import xarray as xr

#Packages for editing the style of our later figures:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap # color ramps
from matplotlib.patches import Patch # legend for final figure

# For the sample data use the two links from figshare (https://ndownloader.figshare.com/files/54896600 AND https://ndownloader.figshare.com/files/55242452) 
# For the data downloads skip to the two notebooks on vegetation-91 and vegetation-92
# Set these up in my local directory (desktop folder earth-analytics)
# Since we are pulling data from zipfiles I will need another library for that
import zipfile 

# STEP 3: Plot NDVI

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Plot the change in NDVI spatially</div></div><div class="callout-body-container callout-body"><p>Complete the following:</p>
<ul>
<li>Select data from 2021 to 2023 (3 years after the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="1"></span>)</li>
<li>Take the temporal mean (over the <strong>date</strong>, not
spatially)</li>
<li>Get the NDVI variable (should be a DataArray, not a Dataset)</li>
<li>Repeat for the data from 2018 to 2020 (3 years before the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span>)</li>
<li>Subtract the 2018-2020 time period <strong>from</strong> the
2021-2023 time period</li>
<li>Plot the result using a <strong>diverging</strong> color map like
<code>cmap=plt.cm.PiYG</code></li>
</ul>
<p>There are different types of color maps for different types of data.
In this case, we want decreases to be a different color from increases,
so we should use a <strong>diverging</strong> color map. Check out
available colormaps in the <a
href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">matplotlib
documentation</a>.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?</div></div><div class="callout-body-container callout-body"><p>For an extra challenge, add the <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary to the plot.</p></div></div>

In [3]:
# Break up these code cells into manageable chunks to 
    # 1) Select data from 2021 to 2023, 
    # 2) Take the temporal mean (over the date, not spatially), 
    # 3) Get the NDVI variable (should be a DataArray, not a Dataset), 
    # 4) Repeat for the data from 2018 to 2020, 
    # 5) Subtract the 2018-2020 time period from the 2021-2023 time period and lastly, 
    # 6) Plot the result using a diverging color map like cmap=plt.cm.PiYG
    # extra 7) Make a boundary (I made a boundary_gdf before we can use it!)

In [4]:
# So looks like I do need to scale my ndvi_stack variable for this
# Scaling the ndvi_stack will produce the NDVI range between [-1,1]
# I'll call this ndvi_scaled

ndvi_scaled = ndvi_stack * 0.0001
ndvi_scaled.attrs["units"] = "NDVI (scaled -1 to 1)"

In [6]:
# Need to import that datetime library again
from datetime import date

In [7]:
# Complete part 1 for 2021-2023 data chunk 
    # 1) Select data from 2021 to 2023
    # We need a new var for this, lets say ndvi_21_23 using the new ndvi_scaled var
    # We just need to specify which slice of dates we want to do our mean with

ndvi_21_23 = ndvi_scaled.sel(date=slice(date(2021, 1, 1), date(2023, 12, 31)))

# Complete part 2 for 2021-2023 data chunk 
    # 2) Take the temporal mean (over the date, not spatially)
    # We will need a new var for this mean value as well
    # Lets match it with our previous var so they match as a group, so mean_21_23

mean_21_23 = ndvi_21_23.mean(dim="date")

# Complete part 3 for 2021-2023 data chunk 
    # 3) Get the NDVI variable (should be a DataArray, not a Dataset)
    # Both ndvi_21_23 and mean_21_23 are already dataarrays (i.e., y, x)


In [8]:
# Complete part 1 for 2018-2020 data chunk 
    # 1) Select data from 2021 to 2023
    # We need a new var for this, lets say ndvi_18_20 using the new ndvi_scaled var
    # We just need to specify which slice of dates we want to do our mean with

ndvi_18_20 = ndvi_scaled.sel(date=slice(date(2018, 1, 1), date(2020, 12, 31)))

# Complete part 2 for 2018-2020 data chunk 
    # 2) Take the temporal mean (over the date, not spatially)
    # We will need a new var for this mean value as well
    # Lets match it with our previous var so they match as a group, so mean_18_20

mean_18_20 = ndvi_18_20.mean(dim="date")

# Complete part 3 for 2018-2020 data chunk 
    # 3) Get the NDVI variable (should be a DataArray, not a Dataset)
    # Both ndvi_18_20 and mean_18_20 are already dataarrays (i.e., y, x)

In [9]:
# Complete part 5) Subtract the 2018-2020 time period from the 2021-2023 time period
# Just take our two mean vars (mean_21_23 and mean_18_20) and substract them from each other
# Lets call this new var ndvi_diff
# If we get a positive value NDVI increased, if we get a negative value NDVI decreased

ndvi_diff = mean_21_23 - mean_18_20

# For when we plot we can also add a description

ndvi_diff.attrs["long_name"] = "NDVI change (2021_2023 minus 2018_2020)"

# call ndvi_diff
ndvi_diff

<xarray.DataArray 'NDVI' (y: 203, x: 382)> Size: 310kB
array([[ 1.2167150e-01,  5.5700034e-02, -3.9428622e-02, ...,
         1.5490457e-02,  1.5490457e-02,  1.7069012e-02],
       [ 6.7580879e-02,  6.9728553e-02, -6.1571598e-03, ...,
         2.4321422e-02,  2.4321422e-02,  2.7740479e-02],
       [-7.5190365e-03, -5.7953000e-03, -1.4154732e-02, ...,
         1.1452377e-02,  2.1297604e-02,  1.5245244e-02],
       ...,
       [-5.4999888e-03, -5.4999888e-03, -7.1904808e-03, ...,
         6.1188638e-04,  6.1188638e-04, -9.5248222e-05],
       [-9.4904751e-03, -9.4904751e-03, -6.5904856e-03, ...,
        -7.6903403e-04, -7.6903403e-04,  9.8285973e-03],
       [-1.3071299e-03, -8.9856833e-03, -5.4357201e-03, ...,
        -1.4976114e-03, -7.2428584e-03, -3.7381053e-04]],
      shape=(203, 382), dtype=float32)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0
Attributes:
    long_name:  NDVI change (2021_2023 minus 2018_2020)

INSIGHT NOTES: So with our NDVI now scaled, this is a bit easier to interpret. We can see in the ARRAY that we have both positive and negative values. That means changes within the Gila boundary are NOT uniform. You will also notice we have e to the nth power on these values, so the changes are rather small roughly between (-.04 and .12) from the displayed values.  

In [12]:
# Complete part 6) Plot the result using a diverging color map like cmap=plt.cm.PiYG
# Complete part 7) Add a boundary

# I thought I might need the other matplotlib libraries for making this figure but they seem overkill for this

# When I looked up diverging colormaps and PiYG code it looks like using vmax is recommended. Using clim=(-vmax, vmax) helps with the vizualization of the NDVI mean diff data
#### Explaing vmax for dummies (me) ####
# When plotting NDVI change with a diverging colormap like 'PiYG', the map needs to know:
    # 1) How “big” a positive value should be before it gets the strongest color (lets say blue)
    # 2) How “big” a negative value should be before it gets the strongest color (lets say purple)
    # In a nutshell -vmax and vmax helps controll the cut off values for this 

# So before we make the map we need to calculate the vmax. This essentially will do the following: 
    # vmax → the maximum absolute NDVI change value to visualize
    # -vmax → the minimum (negative) NDVI change value to visualize
    # This (hopefully) means that the NDVI range from [-1,1] is balanced

# Need a library for calculating this
import numpy as np

# Compute symmetric limit (max absolute value)
vmax = float(np.nanmax(np.abs(ndvi_diff.values)))

print("vmax =", vmax)

vmax = 0.48192140460014343


In [16]:
# Now that we have the vmax issue resolved we can continue with plotting

# NDVI difference map INDENT INDENT INDENT
ndvi_diff_plot = ndvi_diff.hvplot.image(
    x='x', y='y',
    geo=True,
    rasterize=True,
    cmap='PiYG',                 # diverging colormap as directed
    clim=(-vmax, vmax),          # symmetric around 0 (hopefully) with using vmax
    tiles='EsriImagery',
    frame_width=500,
    frame_height=400, 
    title="NDVI change (2021_2023 minus 2018_2020)"
)

# Boundary overlay INDENT INDENT INDENT
boundary_plot = boundary_gdf.hvplot(
    geo=True,
    fill_color=None,
    line_color='black',
    line_width=2,
    frame_width=500,
    frame_height=400, 
)

ndvi_diff_plot * boundary_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'84e46ce1-f4e1-4a6c-a936-73fe1c1cbce5': {'version…

In [17]:
# Lets export this plot
import holoviews as hv

combined_plot = ndvi_diff_plot * boundary_plot

hv.save(combined_plot, 'ndvi_change_map.html')

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [19]:
%store ndvi_scaled ndvi_18_20 ndvi_21_23 ndvi_diff

Stored 'ndvi_scaled' (DataArray)
Stored 'ndvi_18_20' (DataArray)
Stored 'ndvi_21_23' (DataArray)
Stored 'ndvi_diff' (DataArray)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!